Generate map data to be used on a custom Google Map https://www.google.com/mymaps


In [1]:
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
print("Imported libs")

Imported libs


In [18]:
colocs = pl.read_csv("seen_colocs.csv")
colocs

colocationCenter,city,country
str,str,str
"""CFC""","""Chapeco""","""Brazil"""
"""SOF""","""Sofia""","""Bulgaria"""
"""MSP""","""Minneapolis""","""United States"""
"""IST""","""Istanbul""","""Turkey"""
"""MNL""","""Manila""","""Philippines"""
…,…,…
"""BEL""","""Belem""","""Brazil"""
"""PAT""","""Patna""","""India"""
"""MBA""","""Mombasa""","""Kenya"""


In [10]:
world_cities = pl.read_csv("worldcities.csv", ignore_errors=True)
world_cities

city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
str,str,f64,f64,str,str,str,str,str,i64,i64
"""Tokyo""","""Tokyo""",35.687,139.7495,"""Japan""","""JP""","""JPN""","""Tōkyō""","""primary""",37785000,1392685764
"""Jakarta""","""Jakarta""",-6.175,106.8275,"""Indonesia""","""ID""","""IDN""","""Jakarta""","""primary""",33756000,1360771077
"""Delhi""","""Delhi""",28.61,77.23,"""India""","""IN""","""IND""","""Delhi""","""admin""",32226000,1356872604
"""Guangzhou""","""Guangzhou""",23.13,113.26,"""China""","""CN""","""CHN""","""Guangdong""","""admin""",26940000,1156237133
"""Mumbai""","""Mumbai""",19.0761,72.8775,"""India""","""IN""","""IND""","""Mahārāshtra""","""admin""",24973000,1356226629
…,…,…,…,…,…,…,…,…,…,…
"""Al Jabīn""","""Al Jabin""",14.704,43.599,"""Yemen""","""YE""","""YEM""","""Raymah""","""admin""",null,1887910100
"""Nelspruit""","""Nelspruit""",-25.4745,30.9703,"""South Africa""","""ZA""","""ZAF""","""Mpumalanga""","""admin""",null,1710114438
"""Gqeberha""","""Gqeberha""",-33.9681,25.5981,"""South Africa""","""ZA""","""ZAF""","""Eastern Cape""","""""",null,1710000082


In [6]:
data = pl.read_csv("all_combined.csv", null_values=["null"])
data = data.with_columns(
    pl.from_epoch("timestamp", time_unit="s")
)
data

timestamp,balancerId,balancerIp,clientIpAccordingCloudflare,clientCountryAccordingCloudflare,balancerColocationCenter,targetDomain,scheme,httpVersion,tlsVersion,clientCountry,clientCity,clientAsn,clientNetwork,latencyTotal,latencyDNS,latencyTCP,latencyTLS,latencyFirstByte,latencyDownload
datetime[μs],str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,i64,i64,i64,i64,i64
2026-01-21 22:50:36,"""107f211""","""104.16.133.229""","""185.32.190.187""","""PT""","""LIS""","""cloudflare.com""","""https""","""http/2""","""TLSv1.3""","""PT""","""Lisbon""",62416,"""SAMPLING LINE-SERVICOS E INTER…",84,14,3,46,15,6
2026-01-21 22:50:36,"""107f156""","""104.16.133.229""","""185.32.190.187""","""PT""","""LIS""","""cloudflare.com""","""https""","""http/2""","""TLSv1.3""","""PT""","""Lisbon""",62416,"""SAMPLING LINE-SERVICOS E INTER…",78,11,1,18,37,11
2026-01-21 22:50:36,"""107f211""","""104.16.132.229""","""185.32.190.187""","""PT""","""LIS""","""cloudflare.com""","""https""","""http/2""","""TLSv1.3""","""PT""","""Lisbon""",62416,"""SAMPLING LINE-SERVICOS E INTER…",23,2,1,13,5,2
2026-01-21 22:50:37,"""107f179""","""104.16.132.229""","""185.32.190.187""","""PT""","""LIS""","""cloudflare.com""","""https""","""http/2""","""TLSv1.3""","""PT""","""Lisbon""",62416,"""SAMPLING LINE-SERVICOS E INTER…",27,1,1,14,10,1
2026-01-21 22:50:37,"""107f174""","""104.16.132.229""","""185.32.190.187""","""PT""","""LIS""","""cloudflare.com""","""https""","""http/2""","""TLSv1.3""","""PT""","""Lisbon""",62416,"""SAMPLING LINE-SERVICOS E INTER…",29,2,0,18,7,2
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2026-01-28 16:20:35,"""1058f92""","""104.16.133.229""","""179.106.105.87""","""BR""","""GRU""","""cloudflare.com""","""https""","""http/2""","""TLSv1.3""","""BR""","""Belo Horizonte""",52532,"""Speednet Telecomunicacoes Ltda""",730,62,97,100,327,144
2026-01-28 16:20:35,"""728f19""","""104.16.132.229""","""179.106.105.87""","""BR""","""GRU""","""cloudflare.com""","""https""","""http/2""","""TLSv1.3""","""BR""","""Belo Horizonte""",52532,"""Speednet Telecomunicacoes Ltda""",716,45,104,150,272,145
2026-01-28 16:20:36,"""728f183""","""104.16.132.229""","""179.106.105.87""","""BR""","""GRU""","""cloudflare.com""","""https""","""http/2""","""TLSv1.3""","""BR""","""Belo Horizonte""",52532,"""Speednet Telecomunicacoes Ltda""",760,40,128,383,181,28


In [20]:
colocs_with_coords = colocs.join(world_cities, left_on="city", right_on="city_ascii", how="left")
colocs_with_coords = colocs_with_coords.select(["colocationCenter", "city", "country","lat", "lng"]).unique(subset=["colocationCenter"])
colocs_with_coords

colocationCenter,city,country,lat,lng
str,str,str,f64,f64
"""MAO""","""Manaus""","""Brazil""",-3.1189,-60.0217
"""SIN""","""Singapore""","""Singapore""",1.3,103.8
"""DEN""","""Denver""","""United States""",39.762,-104.8758
"""GYD""","""Baku""","""Azerbaijan""",40.3667,49.8352
"""KCH""","""Kuching""","""Malaysia""",1.53,110.33
…,…,…,…,…
"""OSL""","""Oslo""","""Norway""",59.9133,10.7389
"""EZE""","""Buenos Aires""","""Argentina""",-34.6036,-58.3814
"""TBS""","""Tbilisi""","""Georgia""",41.7225,44.7925


In [21]:
missing_colocs = (
    data.select(pl.col("balancerColocationCenter").unique())
    .join(colocs_with_coords, left_on="balancerColocationCenter", right_on="colocationCenter", how="full")
    .filter(pl.col("colocationCenter").is_null())
)
missing_colocs["balancerColocationCenter"].to_list()


[]

Should be empty, otherwise seen_colocs.csv needs updating with the missing colocs

In [22]:
data_grouped_by_colocation = (
    data
    .group_by("balancerColocationCenter")
    .agg(
        pl.col("balancerId").unique().count().alias("balancerCount")
    )
)
with_coords = data_grouped_by_colocation.join(colocs_with_coords, left_on="balancerColocationCenter", right_on="colocationCenter", how="left")
with_coords = with_coords.select(["balancerColocationCenter", "city", "lat", "lng", "balancerCount"])
with_coords


balancerColocationCenter,city,lat,lng,balancerCount
str,str,f64,f64,u32
"""ICN""","""Seoul""",37.5667,126.9833,373
"""KIX""","""Osaka""",34.6939,135.5022,473
"""DEN""","""Denver""",39.762,-104.8758,288
"""RUH""","""Riyadh""",24.65,46.71,9
"""OSL""","""Oslo""",59.9133,10.7389,133
…,…,…,…,…
"""OKC""","""Oklahoma City""",35.4676,-97.5136,21
"""ATH""","""Athens""",37.9842,23.7281,126
"""BUD""","""Budapest""",47.4983,19.0408,87
